In [1]:
import pandas as pd
import json
import os.path
import matplotlib.pyplot as plt
import collections
import matplotlib as m
import copy
import seaborn as sns
import re
import numpy as np
import os
from datetime import datetime

In [58]:
def maybe_parse_time(str):
    try:
        return datetime.strptime(str, '%H:%M:%S.%f')
    except ValueError:
        return False

In [106]:
maybe_parse_time('16:12:12.228744')

datetime.datetime(1900, 1, 1, 16, 12, 12, 228744)

In [107]:
def parse_call_time(line):
    matches = re.findall('([^ ]+) time: ([0-9]+)nanoseconds\s*',line)
    if not matches:
        return False
    elif len(matches) != 1 or len(matches[0]) != 2:
        raise Exception('Uh oh! ' + line, matches)
    
    (fname, ns) = matches[0]
    
    return (fname,int(ns))

In [93]:
def combine(raw_dir, csv_out):
    
    csvs = open(csv_out,'w')
    csvs.write('batch,version,case,local_seq,fname,ns\n')
    
    issues = open('issues.txt','w')
    any_issues = False
    
    for batch in os.listdir(raw_dir):
        for version in os.listdir(os.path.join(raw_dir,batch)):
            for case in os.listdir(os.path.join(raw_dir,batch,version)):
                filename = os.path.join(raw_dir,batch,version,case)
                file = open(filename,'r')
                
                lineno = 0
                local_seq = 0
                for line in file:
                    res = parse_call_time(line)
                    if not res:
                        if not line.startswith('Initializing') and not line.startswith('testing consistency') and not maybe_parse_time(line.strip()):
                            any_issues=True
                            issuestr = f'in {filename} SKIP [{lineno}]: {line[:-1]}'
                            print(issuestr)
                            issues.write(issuestr+'\n')
                        lineno += 1
                        continue
                    lineno += 1
                    (fname,ns) = res
                    
                    csvs.write(f'{global_seq}, {batch}, {version}, {case}, {local_seq}, {fname}, {ns}\n')
                    
                    local_seq+=1
    if any_issues:
        print('Some parsing issues!')
    else:
        print('No parsing issues :)')
        

In [94]:
combine('../output','results0.csv')

No parsing issues :)


In [95]:
df = pd.read_csv('results0.csv')

In [96]:
df

,batch,version,case,local_seq,fname,ns
35717,read-perf,basic,256_KB,0,getattr,1932816
35717,read-perf,basic,256_KB,1,getattr,2536427
35717,read-perf,basic,256_KB,2,getattr,1578917
35717,read-perf,basic,256_KB,3,getattr,1353814
35717,read-perf,basic,256_KB,4,open,18828004
...,...,...,...,...,...,...
35717,write-perf,perf,64_B,898,getattr,1426214
35717,write-perf,perf,64_B,899,read,4900
35717,write-perf,perf,64_B,900,write,5434755
35717,write-perf,perf,64_B,901,release,2913229


In [103]:
faceted = df.drop('local_seq',axis=1).groupby(['batch','version','case','fname'])

In [105]:
faceted.mean().to_csv('mean.csv')
faceted.min().to_csv('min.csv')
faceted.max().to_csv('max.csv')
faceted.median().to_csv('median.csv')
faceted.std().to_csv('sd.csv')